# 03 — Ablation study on Graph3 (Goodbooks-10k GNN Recsys)

This notebook answers a simple but crucial question:

> **Which relation groups in Graph3 actually improve recommendation quality?**

We already have a strong **Graph3** constructed in a unified node-id space:
- **Users**, **Books**, and content/context nodes (**Tags**, **Authors**, **Language**, **Year bins**)
- Weighted edges:
  - user ↔ book (TRAIN only)
  - book ↔ tag (weight = log1p(tag_count))
  - book ↔ book (TF-IDF cosine similarity, top-K neighbors, pruned by min similarity)
  - book ↔ author (1.0)
  - book ↔ language (1.0)
  - book ↔ year_bin (1.0)

The goal is **not** to redesign the graph anymore (we consider it maxed out with available Goodbooks signals),
but to **quantify the contribution** of each relation group via systematic ablations.

## Task setup

- **Model:** LightGCN (3 layers, embedding dim = 64)  
- **Loss:** BPR with negative sampling (negatives exclude seen TRAIN positives)
- **Split:** leave-one-out (LOO)
  - for each user: 1 validation item, 1 test item
- **Metrics:** Hit@K / NDCG@K (K ∈ {10, 20, 50})

## What “ablation” means here

We start from the FULL Graph3 and create variants by removing a relation group:

- **−book_sim**: remove book↔book similarity edges  
- **−book_tag**: remove book↔tag edges  
- **−book_author**: remove book↔author edges  
- **−book_lang**: remove book↔language edges  
- **−book_year**: remove book↔year_bin edges  

We also run useful baselines:
- **ONLY_user_book**: only the user↔book bipartite graph
- **user_book+tag**: bipartite + tag relations

Each variant rebuilds the normalized adjacency and trains LightGCN with **the same training loop and eval**.

## Expected outcome

A final table ranking variants by **TEST NDCG@10**, plus deltas relative to FULL.

This tells us:
1) which relations are true “signal” vs “noise”,  
2) what to emphasize in the next notebooks (HeteroGNN / HGT / sampling / attention),  
3) which components are safe to drop for faster training without losing quality.

## Loaded bundle (already prepared)

We load a saved Graph3 bundle from:

`D:/ML/GNN/graph_recsys/artifacts/v2_proper/graph3_bundle`

It contains:
- sparse normalized adjacency `A_norm`
- raw edges `edge_index`, weights `edge_w`
- typed relations `edge_type` and mapping `rel2id`
- fixed LOO splits `train_ui / val_ui / test_ui`
- offsets and vocabularies for reproducibility

In [1]:
# ============================
# Cell 1: Imports + seed
# ============================
import os
import json
import time
import math
import random
from pathlib import Path

import numpy as np
import pandas as pd
import torch
from tqdm.auto import tqdm

In [2]:
# ============================
# Cell 1b: Reproducibility
# ============================
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("DEVICE:", DEVICE)

DEVICE: cuda


In [3]:
# ============================
# Cell 2: Paths + load bundle
# ============================
PROJECT_ROOT = Path(r"D:/ML/GNN/graph_recsys")
ARTIFACTS = PROJECT_ROOT / "artifacts" / "v2_proper"
BUNDLE_DIR = ARTIFACTS / "graph3_bundle"

assert BUNDLE_DIR.exists(), f"Missing bundle dir: {BUNDLE_DIR}"
print("BUNDLE_DIR:", BUNDLE_DIR)

g = torch.load(BUNDLE_DIR / "graph3_state.pt", map_location="cpu")
z = np.load(BUNDLE_DIR / "splits_ui.npz", allow_pickle=True)

A_norm_full = g["A_norm"]                 # sparse normalized adjacency (full Graph3)
edge_index_full = g["edge_index"]         # [2, E]
edge_w_full = g["edge_w"]                 # [E]
edge_type_full = g["edge_type"]           # [E]
rel2id = g["rel2id"]                      # dict[str,int]
offsets = g["offsets"]
num_nodes = int(g["num_nodes"])

train_ui = z["train_ui"].astype(np.int64) # [N_train, 2] (user_idx, book_idx)
val_ui   = z["val_ui"].astype(np.int64)   # [U, 2] LOO
test_ui  = z["test_ui"].astype(np.int64)  # [U, 2] LOO
U = int(z["U"]); B = int(z["B"])

print(f"Loaded: num_nodes={num_nodes}, E={edge_index_full.shape[1]}, U={U}, B={B}, rels={len(rel2id)}")
print("Relations:", rel2id)

BUNDLE_DIR: D:\ML\GNN\graph_recsys\artifacts\v2_proper\graph3_bundle


C:\Users\555\AppData\Local\Temp\ipykernel_15212\2403994732.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  g = torch.load(BUNDLE_DIR / "graph3_state.pt", map_location="

Loaded: num_nodes=74285, E=11450076, U=53398, B=9999, rels=11
Relations: {'user_book': 0, 'book_user': 1, 'book_tag': 2, 'tag_book': 3, 'book_book_sim': 4, 'book_author': 5, 'author_book': 6, 'book_lang': 7, 'lang_book': 8, 'book_year': 9, 'year_book': 10}


In [4]:
# ============================
# Cell 3: Helpers (filter edges + rebuild normalized adjacency)
# ============================
def filter_edges_by_rel(edge_index, edge_w, edge_type, keep_rel_ids):
    """
    Возвращает подграф, содержащий только ребра с типами из keep_rel_ids.
    """
    keep_rel_ids = set(int(x) for x in keep_rel_ids)
    mask = torch.zeros_like(edge_type, dtype=torch.bool)
    for rid in keep_rel_ids:
        mask |= (edge_type == rid)

    ei = edge_index[:, mask]
    ew = edge_w[mask]
    et = edge_type[mask]
    return ei, ew, et

def build_sparse_norm(edge_index, edge_w, num_nodes):
    """
    Symmetric norm: D^{-1/2} A D^{-1/2}
    where degree is weighted sum over outgoing edges (row).
    Assumes graph is already undirected by construction (we stored both directions).
    """
    row = edge_index[0]
    col = edge_index[1]
    val = edge_w.float()

    deg = torch.zeros(num_nodes, dtype=torch.float32)
    deg.scatter_add_(0, row, val)
    deg = torch.clamp(deg, min=1e-12)

    inv_sqrt = deg.pow(-0.5)
    norm_val = inv_sqrt[row] * val * inv_sqrt[col]

    A = torch.sparse_coo_tensor(edge_index, norm_val, (num_nodes, num_nodes), dtype=torch.float32).coalesce()
    return A

# sanity check: rebuild should match shape
A_check = build_sparse_norm(edge_index_full, edge_w_full, num_nodes)
print("A_check:", tuple(A_check.shape), "nnz:", int(A_check._nnz()))

A_check: (74285, 74285) nnz: 11260518


In [5]:
# ============================
# Cell 4: LightGCN model
# ============================
class LightGCN(torch.nn.Module):
    def __init__(self, num_nodes, emb_dim=64, n_layers=3):
        super().__init__()
        self.num_nodes = num_nodes
        self.emb_dim = emb_dim
        self.n_layers = n_layers
        self.emb = torch.nn.Embedding(num_nodes, emb_dim)
        torch.nn.init.normal_(self.emb.weight, std=0.1)

    def propagate(self, A_norm):
        """
        A_norm: torch sparse COO [N, N] on DEVICE
        returns: final embeddings [N, D]
        """
        x0 = self.emb.weight
        xs = [x0]
        x = x0
        for _ in range(self.n_layers):
            x = torch.sparse.mm(A_norm, x)
            xs.append(x)
        x_out = torch.stack(xs, dim=0).mean(dim=0)
        return x_out

In [6]:
# ============================
# Cell 5: Train positives + negative sampling
# ============================
from collections import defaultdict

train_pos = defaultdict(set)
for u, i in train_ui:
    train_pos[int(u)].add(int(i))

val_gt  = {int(u): int(i) for u, i in val_ui}
test_gt = {int(u): int(i) for u, i in test_ui}

print("train_pos users:", len(train_pos), "val_gt:", len(val_gt), "test_gt:", len(test_gt))

train_pos users: 53398 val_gt: 53398 test_gt: 53398


In [7]:
# ============================
# Cell 5b: vectorized negative sampling (per batch)
# ============================
def sample_negatives(users_np, B, train_pos, n_neg=1, max_tries=50):
    """
    Для каждого user выбираем n_neg отрицательных items, избегая train positives.
    Возвращает shape [len(users), n_neg]
    """
    users_np = users_np.astype(np.int64)
    out = np.empty((len(users_np), n_neg), dtype=np.int64)

    for idx, u in enumerate(users_np):
        s = train_pos[int(u)]
        for k in range(n_neg):
            # rejection sampling
            for _ in range(max_tries):
                j = np.random.randint(0, B)
                if j not in s:
                    out[idx, k] = j
                    break
            else:
                # fallback if user почти всё видел
                j = np.random.randint(0, B)
                out[idx, k] = j
    return out

In [8]:
# ============================
# Cell 6: Ranking metrics for LOO (Hit@K, NDCG@K)
# ============================
def hit_ndcg_at_k(scores, gt_item, k):
    """
    scores: 1D torch tensor [B] (bigger = better)
    gt_item: int
    """
    topk = torch.topk(scores, k=k).indices.cpu().numpy()
    if gt_item in topk:
        rank = int(np.where(topk == gt_item)[0][0]) + 1  # 1-based
        hit = 1.0
        ndcg = 1.0 / np.log2(rank + 1)
    else:
        hit = 0.0
        ndcg = 0.0
    return hit, ndcg

@torch.no_grad()
def evaluate_loo(emb_all, U, B, offsets, gt_dict, train_pos, Ks=(10,20,50), batch_users=1024):
    """
    emb_all: [num_nodes, D] on DEVICE
    gt_dict: {user_idx: true_item_idx}
    Excludes train positives from ranking by setting -inf.
    """
    user_off = int(offsets["user_offset"])
    book_off = int(offsets["book_offset"])

    # item embeddings for dot-product scoring
    item_emb = emb_all[book_off:book_off+B]  # [B, D]

    hits = {k: 0.0 for k in Ks}
    ndcgs = {k: 0.0 for k in Ks}
    users = np.array(sorted(gt_dict.keys()), dtype=np.int64)

    for start in range(0, len(users), batch_users):
        u_batch = users[start:start+batch_users]
        u_emb = emb_all[user_off + torch.from_numpy(u_batch).to(DEVICE)]  # [bs, D]
        scores = u_emb @ item_emb.T  # [bs, B]

        # mask train positives
        for row_idx, u in enumerate(u_batch):
            seen = list(train_pos[int(u)])
            if seen:
                scores[row_idx, torch.tensor(seen, device=DEVICE)] = -1e9

        # compute metrics
        for row_idx, u in enumerate(u_batch):
            gt = int(gt_dict[int(u)])
            s = scores[row_idx]
            for k in Ks:
                h, n = hit_ndcg_at_k(s, gt, k)
                hits[k] += h
                ndcgs[k] += n

    n = len(users)
    out = {}
    for k in Ks:
        out[f"Hit@{k}"] = hits[k] / n
        out[f"NDCG@{k}"] = ndcgs[k] / n
    return out

In [11]:
# ============================
# Cell 7: Train loop (BPR) - STABLE MODE
# - propagate is recomputed per batch to avoid "backward second time" error
# ============================
def bpr_loss(u_emb, pos_emb, neg_emb):
    pos_scores = (u_emb * pos_emb).sum(dim=1)
    neg_scores = (u_emb * neg_emb).sum(dim=1)
    return -torch.log(torch.sigmoid(pos_scores - neg_scores) + 1e-12).mean()

def train_one_run(
    A_norm_cpu,
    run_name,
    emb_dim=64,
    n_layers=3,
    lr=1e-3,
    epochs=20,
    batch_size=200_000,
    n_neg=1,
    eval_every=1,
    patience=8,
):
    """
    STABLE training:
    - A_norm moved to DEVICE once
    - for each batch: propagate -> loss -> backward -> step
    This avoids the autograd error from reusing the same graph across multiple backward passes.
    """
    A_norm = A_norm_cpu.to(DEVICE)

    model = LightGCN(num_nodes=num_nodes, emb_dim=emb_dim, n_layers=n_layers).to(DEVICE)
    opt = torch.optim.Adam(model.parameters(), lr=lr)

    best_val = -1.0
    best_state = None
    bad = 0

    n_train = train_ui.shape[0]
    idx_all = np.arange(n_train)

    history = []

    user_off = int(offsets["user_offset"])
    book_off = int(offsets["book_offset"])

    for ep in range(1, epochs + 1):
        np.random.shuffle(idx_all)

        model.train()
        total_loss = 0.0
        steps = 0

        for s in range(0, n_train, batch_size):
            batch_idx = idx_all[s:s + batch_size]
            batch = train_ui[batch_idx]  # [bs, 2]

            users_np = batch[:, 0]
            pos_np   = batch[:, 1]
            neg_np   = sample_negatives(users_np, B, train_pos, n_neg=n_neg)[:, 0]

            u = torch.from_numpy(users_np).to(DEVICE)
            i_pos = torch.from_numpy(pos_np).to(DEVICE)
            i_neg = torch.from_numpy(neg_np).to(DEVICE)

            # --- STABLE: recompute propagate per batch (fresh autograd graph) ---
            emb_all = model.propagate(A_norm)

            u_emb = emb_all[user_off + u]
            pos_emb = emb_all[book_off + i_pos]
            neg_emb = emb_all[book_off + i_neg]

            loss = bpr_loss(u_emb, pos_emb, neg_emb)

            opt.zero_grad(set_to_none=True)
            loss.backward()
            opt.step()

            total_loss += float(loss.item())
            steps += 1

        avg_loss = total_loss / max(1, steps)

        # ---- eval ----
        if ep % eval_every == 0:
            model.eval()
            with torch.no_grad():
                emb_all_eval = model.propagate(A_norm)

            val_metrics = evaluate_loo(emb_all_eval, U, B, offsets, val_gt, train_pos, Ks=(10, 20, 50))
            score = val_metrics["NDCG@10"]

            row = {"run": run_name, "epoch": ep, "loss": avg_loss, **val_metrics}
            history.append(row)

            print(f"[{run_name}] ep={ep:03d} loss={avg_loss:.4f} | "
                  f"Hit@10={val_metrics['Hit@10']:.5f} NDCG@10={val_metrics['NDCG@10']:.5f}")

            # early stopping
            if score > best_val + 1e-6:
                best_val = score
                best_state = {"model": model.state_dict(), "epoch": ep, "val_metrics": val_metrics}
                bad = 0
            else:
                bad += 1
                if bad >= patience:
                    print(f"[{run_name}] Early stop at epoch {ep} (best val NDCG@10={best_val:.5f})")
                    break

    return best_state, pd.DataFrame(history)

In [12]:
# ============================
# Cell 7.5: SMOKE TEST (fast debug run)
# - цель: проверить, что filter -> rebuild A_norm -> train loop -> eval работает
# - режем граф и train edges, делаем 2-3 эпохи
# ============================

FAST_DEBUG = True 

if FAST_DEBUG:
    # 1) берём более лёгкую конфигурацию: только user-book + book-tag
    keep_rel_ids = (
        [rel2id["user_book"], rel2id["book_user"]] +
        [rel2id["book_tag"], rel2id["tag_book"]]
    )

    # 2) фильтруем edges
    ei_dbg, ew_dbg, et_dbg = filter_edges_by_rel(
        edge_index_full, edge_w_full, edge_type_full, keep_rel_ids
    )
    print("DEBUG: filtered E =", int(ei_dbg.shape[1]))

    # 3) дополнительно режем число рёбер (чтобы быстро)
    # ВАЖНО: берём первые N рёбер — этого достаточно для smoke теста
    MAX_E = 1_500_000
    if ei_dbg.shape[1] > MAX_E:
        ei_dbg = ei_dbg[:, :MAX_E]
        ew_dbg = ew_dbg[:MAX_E]
        et_dbg = et_dbg[:MAX_E]
    print("DEBUG: used E =", int(ei_dbg.shape[1]))

    # 4) rebuild A_norm
    A_dbg = build_sparse_norm(ei_dbg, ew_dbg, num_nodes)
    print("DEBUG: A_dbg nnz =", int(A_dbg._nnz()))

    # 5) режем train edges (чтобы одна эпоха быстро)
    # сохраним оригинал и временно заменим train_ui
    _train_ui_orig = train_ui.copy()

    MAX_TRAIN_EDGES = 500_000
    if train_ui.shape[0] > MAX_TRAIN_EDGES:
        train_ui = train_ui[:MAX_TRAIN_EDGES].copy()
    print("DEBUG: train_ui =", train_ui.shape)

    # 6) короткий прогон (2-3 эпохи)
    best_state_dbg, hist_dbg = train_one_run(
        A_norm_cpu=A_dbg,
        run_name="SMOKE_user_book+tag",
        emb_dim=64,
        n_layers=2,        
        lr=1e-3,
        epochs=3,            
        batch_size=100_000,  
        n_neg=1,
        patience=10         
    )

    print("\n[SMOKE DONE] last hist row:")
    display(hist_dbg.tail(1))

    # 7) возвращаем train_ui на место
    train_ui = _train_ui_orig
    print("[OK] train_ui restored:", train_ui.shape)

else:
    print("FAST_DEBUG=False -> skip smoke test")

DEBUG: filtered E = 10852488
DEBUG: used E = 1500000
DEBUG: A_dbg nnz = 1500000
DEBUG: train_ui = (500000, 2)
[SMOKE_user_book+tag] ep=001 loss=7.4679 | Hit@10=0.00097 NDCG@10=0.00043
[SMOKE_user_book+tag] ep=002 loss=7.4290 | Hit@10=0.00105 NDCG@10=0.00046
[SMOKE_user_book+tag] ep=003 loss=7.4022 | Hit@10=0.00105 NDCG@10=0.00046

[SMOKE DONE] last hist row:


,run,epoch,loss,Hit@10,NDCG@10,Hit@20,NDCG@20,Hit@50,NDCG@50
2,SMOKE_user_book+tag,3,7.402205,0.001049,0.000459,0.001966,0.000687,0.005356,0.001351


[OK] train_ui restored: (500000, 2)


In [13]:
# ============================
# Cell 7.6: Final checklist before full ablations
# ============================

print("A_norm_full:", tuple(A_norm_full.shape), "nnz:", int(A_norm_full._nnz()))
print("edge_index_full:", tuple(edge_index_full.shape))
print("edge_w_full:", tuple(edge_w_full.shape), "finite:", bool(torch.isfinite(edge_w_full).all().item()))
print("edge_type_full:", tuple(edge_type_full.shape), "n_rels:", len(rel2id))
print("train_ui:", train_ui.shape, "val_ui:", val_ui.shape, "test_ui:", test_ui.shape)
print("U,B:", U, B, "| num_nodes:", num_nodes)

# базовые sanity
assert edge_index_full.shape[1] == edge_w_full.numel() == edge_type_full.numel()
assert int(edge_index_full.max()) < int(num_nodes)
assert int(edge_index_full.min()) >= 0
assert train_ui.shape[1] == 2 and val_ui.shape[1] == 2 and test_ui.shape[1] == 2

print("\n[OK] Ready for full ablations ✅")

A_norm_full: (74285, 74285) nnz: 11260518
edge_index_full: (2, 11450076)
edge_w_full: (11450076,) finite: True
edge_type_full: (11450076,) n_rels: 11
train_ui: (500000, 2) val_ui: (53398, 2) test_ui: (53398, 2)
U,B: 53398 9999 | num_nodes: 74285

[OK] Ready for full ablations ✅


In [14]:
# ============================
# Cell 7.7: FIX - reload FULL splits from bundle
# ============================
z = np.load(BUNDLE_DIR / "splits_ui.npz", allow_pickle=True)

train_ui = z["train_ui"].astype(np.int64)
val_ui   = z["val_ui"].astype(np.int64)
test_ui  = z["test_ui"].astype(np.int64)

U = int(z["U"]); B = int(z["B"])

print("Reloaded splits:")
print("train_ui:", train_ui.shape)
print("val_ui:", val_ui.shape)
print("test_ui:", test_ui.shape)
print("U,B:", U, B)

Reloaded splits:
train_ui: (4926384, 2)
val_ui: (53398, 2)
test_ui: (53398, 2)
U,B: 53398 9999


In [15]:
# ============================
# Cell 8: Define ablations (night run)
# ============================
groups = {
    "user_book":   [rel2id["user_book"], rel2id["book_user"]],
    "book_tag":    [rel2id["book_tag"], rel2id["tag_book"]],
    "book_sim":    [rel2id["book_book_sim"]],
    "book_author": [rel2id["book_author"], rel2id["author_book"]],
    "book_lang":   [rel2id["book_lang"], rel2id["lang_book"]],
    "book_year":   [rel2id["book_year"], rel2id["year_book"]],
}

all_rel_ids = sorted(set(int(v) for v in rel2id.values()))

ablations = [
    ("FULL", all_rel_ids),
    ("ONLY_user_book", groups["user_book"]),
    ("user_book+tag",  groups["user_book"] + groups["book_tag"]),
    ("-book_sim",      [rid for rid in all_rel_ids if rid not in groups["book_sim"]]),
]

pd.DataFrame({"variant":[a[0] for a in ablations], "n_rel_ids":[len(a[1]) for a in ablations]})

,variant,n_rel_ids
0,FULL,11
1,ONLY_user_book,2
2,user_book+tag,4
3,-book_sim,10


In [16]:
# ============================
# Cell 9: Run ablations (tqdm progress + skip + save)
# ============================
import math
from tqdm.auto import tqdm

RESULTS_DIR = ARTIFACTS / "ablation_runs"
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

# ---- Overwrite train_one_run with tqdm-enabled STABLE loop ----
def train_one_run(
    A_norm_cpu,
    run_name,
    emb_dim=64,
    n_layers=3,
    lr=1e-3,
    epochs=20,
    batch_size=300_000,
    n_neg=1,
    eval_every=1,
    patience=5,
):
    """
    STABLE training (propagate per batch) + tqdm progress bar over batches.
    This is slower but robust and won't crash with autograd 'backward second time'.
    """
    A_norm = A_norm_cpu.to(DEVICE)

    model = LightGCN(num_nodes=num_nodes, emb_dim=emb_dim, n_layers=n_layers).to(DEVICE)
    opt = torch.optim.Adam(model.parameters(), lr=lr)

    best_val = -1.0
    best_state = None
    bad = 0
    history = []

    n_train = train_ui.shape[0]
    idx_all = np.arange(n_train)

    user_off = int(offsets["user_offset"])
    book_off = int(offsets["book_offset"])

    for ep in range(1, epochs + 1):
        np.random.shuffle(idx_all)

        model.train()
        total_loss = 0.0
        steps = 0

        n_batches = math.ceil(n_train / batch_size)
        pbar = tqdm(range(0, n_train, batch_size), total=n_batches,
                    desc=f"{run_name} | ep {ep:03d}", leave=False)

        for s in pbar:
            batch_idx = idx_all[s:s + batch_size]
            batch = train_ui[batch_idx]  # [bs, 2]

            users_np = batch[:, 0]
            pos_np   = batch[:, 1]
            neg_np   = sample_negatives(users_np, B, train_pos, n_neg=n_neg)[:, 0]

            u = torch.from_numpy(users_np).to(DEVICE)
            i_pos = torch.from_numpy(pos_np).to(DEVICE)
            i_neg = torch.from_numpy(neg_np).to(DEVICE)

            # STABLE: fresh autograd graph each batch
            emb_all = model.propagate(A_norm)

            u_emb = emb_all[user_off + u]
            pos_emb = emb_all[book_off + i_pos]
            neg_emb = emb_all[book_off + i_neg]

            loss = bpr_loss(u_emb, pos_emb, neg_emb)

            opt.zero_grad(set_to_none=True)
            loss.backward()
            opt.step()

            total_loss += float(loss.item())
            steps += 1
            pbar.set_postfix(loss=total_loss / max(1, steps))

        avg_loss = total_loss / max(1, steps)

        # ---- eval ----
        if ep % eval_every == 0:
            model.eval()
            with torch.no_grad():
                emb_all_eval = model.propagate(A_norm)

            val_metrics = evaluate_loo(emb_all_eval, U, B, offsets, val_gt, train_pos, Ks=(10, 20, 50))
            score = val_metrics["NDCG@10"]

            row = {"run": run_name, "epoch": ep, "loss": avg_loss, **val_metrics}
            history.append(row)

            print(f"[{run_name}] ep={ep:03d} loss={avg_loss:.4f} | "
                  f"Hit@10={val_metrics['Hit@10']:.5f} NDCG@10={val_metrics['NDCG@10']:.5f}")

            if score > best_val + 1e-6:
                best_val = score
                best_state = {"model": model.state_dict(), "epoch": ep, "val_metrics": val_metrics}
                bad = 0
            else:
                bad += 1
                if bad >= patience:
                    print(f"[{run_name}] Early stop at epoch {ep} (best val NDCG@10={best_val:.5f})")
                    break

    return best_state, pd.DataFrame(history)


# ---- Run variants ----
all_results = []
all_hist = []

for variant, keep_rel_ids in ablations:
    print("\n" + "="*90)
    print("Variant:", variant, "| keep_rel_ids:", len(keep_rel_ids))

    # SKIP if already computed
    sum_path = RESULTS_DIR / f"summary_{variant}.json"
    hist_path = RESULTS_DIR / f"hist_{variant}.csv"
    if sum_path.exists() and hist_path.exists():
        print(f"[SKIP] {variant} already computed")
        # optionally load existing summary for the final table
        with open(sum_path, "r", encoding="utf-8") as f:
            all_results.append(json.load(f))
        continue

    # 1) filter edges
    ei, ew, et = filter_edges_by_rel(edge_index_full, edge_w_full, edge_type_full, keep_rel_ids)
    print("Filtered E:", int(ei.shape[1]))

    # 2) rebuild A_norm
    A_var = build_sparse_norm(ei, ew, num_nodes)
    print("A_var nnz:", int(A_var._nnz()))

    # 3) train
    best_state, hist_df = train_one_run(
        A_norm_cpu=A_var,
        run_name=variant,
        emb_dim=64,
        n_layers=3,
        lr=1e-3,
        epochs=20,
        batch_size=300_000,   
        n_neg=1,
        patience=5
    )

    # 4) evaluate on TEST at best epoch
    model = LightGCN(num_nodes=num_nodes, emb_dim=64, n_layers=3).to(DEVICE)
    model.load_state_dict(best_state["model"])

    model.eval()
    with torch.no_grad():
        emb_all = model.propagate(A_var.to(DEVICE))

    test_metrics = evaluate_loo(emb_all, U, B, offsets, test_gt, train_pos, Ks=(10, 20, 50))

    row = {
        "variant": variant,
        "best_epoch": int(best_state["epoch"]),
        **{f"val_{k}": float(v) for k, v in best_state["val_metrics"].items()},
        **{f"test_{k}": float(v) for k, v in test_metrics.items()},
        "E": int(ei.shape[1]),
        "nnz": int(A_var._nnz()),
    }

    all_results.append(row)
    all_hist.append(hist_df)

    # save
    hist_df.to_csv(hist_path, index=False)
    with open(sum_path, "w", encoding="utf-8") as f:
        json.dump(row, f, ensure_ascii=False, indent=2)

    print(f"[DONE] {variant} | TEST Hit@10={test_metrics['Hit@10']:.5f} NDCG@10={test_metrics['NDCG@10']:.5f}")


# ---- Final summary table ----
res = pd.DataFrame(all_results)
if len(res) > 0:
    res = res.sort_values("test_NDCG@10", ascending=False).reset_index(drop=True)

cols = [
    "variant", "E", "nnz", "best_epoch",
    "val_Hit@10", "val_NDCG@10",
    "test_Hit@10", "test_NDCG@10",
    "test_Hit@20", "test_NDCG@20",
    "test_Hit@50", "test_NDCG@50",
]
display(res[cols])

out_path = RESULTS_DIR / "ablation_summary.csv"
res.to_csv(out_path, index=False)
print("[OK] saved:", out_path)


Variant: FULL | keep_rel_ids: 11
Filtered E: 11450076
A_var nnz: 11260518


FULL | ep 001:   0%|          | 0/17 [00:00<?, ?it/s]

[FULL] ep=001 loss=0.6927 | Hit@10=0.01157 NDCG@10=0.00593


FULL | ep 002:   0%|          | 0/17 [00:00<?, ?it/s]

[FULL] ep=002 loss=0.6883 | Hit@10=0.04674 NDCG@10=0.02416


...

[-book_sim] ep=019 loss=0.3678 | Hit@10=0.05206 NDCG@10=0.02872


-book_sim | ep 020:   0%|          | 0/17 [00:00<?, ?it/s]

[-book_sim] ep=020 loss=0.3627 | Hit@10=0.05309 NDCG@10=0.02905
[DONE] -book_sim | TEST Hit@10=0.05328 NDCG@10=0.02902


,variant,E,nnz,best_epoch,val_Hit@10,val_NDCG@10,test_Hit@10,test_NDCG@10,test_Hit@20,test_NDCG@20,test_Hit@50,test_NDCG@50
0,FULL,11450076,11260518,20,0.054815,0.029770,0.055152,0.029817,0.082887,0.036756,0.142945,0.048596
1,ONLY_user_book,9852768,9852768,20,0.054515,0.029567,0.054983,0.029727,0.082924,0.036715,0.142833,0.048543
2,user_book+tag,10852488,10852482,20,0.053598,0.029399,0.053878,0.029497,0.082194,0.036590,0.141110,0.048221
3,-book_sim,10918914,10918894,20,0.053092,0.029047,0.053279,0.029020,0.080958,0.035954,0.140473,0.047699


[OK] saved: D:\ML\GNN\graph_recsys\artifacts\v2_proper\ablation_runs\ablation_summary.csv


In [17]:
# ============================
# Cell 10: Summarize ablations
# ============================
res = pd.DataFrame(all_results)

# сортируем по test NDCG@10
res = res.sort_values("test_NDCG@10", ascending=False).reset_index(drop=True)

cols = [
    "variant", "E", "nnz", "best_epoch",
    "val_Hit@10", "val_NDCG@10", "test_Hit@10", "test_NDCG@10",
    "test_Hit@20", "test_NDCG@20", "test_Hit@50", "test_NDCG@50"
]
display(res[cols])

,variant,E,nnz,best_epoch,val_Hit@10,val_NDCG@10,test_Hit@10,test_NDCG@10,test_Hit@20,test_NDCG@20,test_Hit@50,test_NDCG@50
0,FULL,11450076,11260518,20,0.054815,0.029770,0.055152,0.029817,0.082887,0.036756,0.142945,0.048596
1,ONLY_user_book,9852768,9852768,20,0.054515,0.029567,0.054983,0.029727,0.082924,0.036715,0.142833,0.048543
2,user_book+tag,10852488,10852482,20,0.053598,0.029399,0.053878,0.029497,0.082194,0.036590,0.141110,0.048221
3,-book_sim,10918914,10918894,20,0.053092,0.029047,0.053279,0.029020,0.080958,0.035954,0.140473,0.047699


In [18]:
# ============================
# Cell 11: Save final results table
# ============================
out_path = RESULTS_DIR / "ablation_summary.csv"
res.to_csv(out_path, index=False)
print("[OK] saved:", out_path)

[OK] saved: D:\ML\GNN\graph_recsys\artifacts\v2_proper\ablation_runs\ablation_summary.csv


In [20]:
# ============================
# Define BIG ablations (full list)
# ============================

import json, math
import numpy as np
import pandas as pd
import torch
from tqdm.auto import tqdm


groups = {
    "user_book":    [rel2id["user_book"], rel2id["book_user"]],
    "book_tag":     [rel2id["book_tag"], rel2id["tag_book"]],
    "book_sim":     [rel2id["book_book_sim"]],
    "book_author":  [rel2id["book_author"], rel2id["author_book"]],
    "book_lang":    [rel2id["book_lang"], rel2id["lang_book"]],
    "book_year":    [rel2id["book_year"], rel2id["year_book"]],
}

all_rel_ids = sorted(set(int(v) for v in rel2id.values()))

# Большая абляция: удаляем по одному "блоку" отношений
ablations_big = [
    ("FULL", all_rel_ids),

    ("-book_sim",   [rid for rid in all_rel_ids if rid not in groups["book_sim"]]),
    ("-book_tag",   [rid for rid in all_rel_ids if rid not in groups["book_tag"]]),
    ("-book_author",[rid for rid in all_rel_ids if rid not in groups["book_author"]]),
    ("-book_lang",  [rid for rid in all_rel_ids if rid not in groups["book_lang"]]),
    ("-book_year",  [rid for rid in all_rel_ids if rid not in groups["book_year"]]),

    # полезные sanity / минималки:
    ("ONLY_user_book", groups["user_book"]),
    ("user_book+tag",  groups["user_book"] + groups["book_tag"]),
]

pd.DataFrame(
    {"variant":[a[0] for a in ablations_big],
     "n_rel_ids":[len(a[1]) for a in ablations_big]}
)

,variant,n_rel_ids
0,FULL,11
1,-book_sim,10
2,-book_tag,9
3,-book_author,9
4,-book_lang,9
5,-book_year,9
6,ONLY_user_book,2
7,user_book+tag,4


In [22]:
# ============================
# Run BIG ablations (tqdm + skip + save) [FIXED]
# ============================

RESULTS_DIR = ARTIFACTS / "ablation_runs"
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

# ----------------------------
# Train loop: STABLE + tqdm
# ----------------------------
def train_one_run(
    A_norm_cpu,
    run_name,
    emb_dim=64,
    n_layers=3,
    lr=1e-3,
    epochs=12,             # FAST for big ablations
    batch_size=300_000,    # your sweet spot
    n_neg=1,
    eval_every=1,
    patience=3,            # FAST early stop
):
    """
    STABLE training:
    - propagate per batch (robust)
    - tqdm shows per-batch progress
    """
    A_norm = A_norm_cpu.to(DEVICE)

    model = LightGCN(num_nodes=num_nodes, emb_dim=emb_dim, n_layers=n_layers).to(DEVICE)
    opt = torch.optim.Adam(model.parameters(), lr=lr)

    best_val = -1.0
    best_state = None
    bad = 0
    history = []

    n_train = train_ui.shape[0]
    idx_all = np.arange(n_train)

    user_off = int(offsets["user_offset"])
    book_off = int(offsets["book_offset"])

    for ep in range(1, epochs + 1):
        np.random.shuffle(idx_all)

        model.train()
        total_loss = 0.0
        steps = 0

        n_batches = math.ceil(n_train / batch_size)
        pbar = tqdm(
            range(0, n_train, batch_size),
            total=n_batches,
            desc=f"{run_name} | ep {ep:03d}",
            leave=False
        )

        for s in pbar:
            batch_idx = idx_all[s:s + batch_size]
            batch = train_ui[batch_idx]  # [bs, 2]

            users_np = batch[:, 0]
            pos_np   = batch[:, 1]
            neg_np   = sample_negatives(users_np, B, train_pos, n_neg=n_neg)[:, 0]

            u = torch.from_numpy(users_np).to(DEVICE)
            i_pos = torch.from_numpy(pos_np).to(DEVICE)
            i_neg = torch.from_numpy(neg_np).to(DEVICE)

            # STABLE: fresh autograd graph per batch
            emb_all = model.propagate(A_norm)

            u_emb   = emb_all[user_off + u]
            pos_emb = emb_all[book_off + i_pos]
            neg_emb = emb_all[book_off + i_neg]

            loss = bpr_loss(u_emb, pos_emb, neg_emb)

            opt.zero_grad(set_to_none=True)
            loss.backward()
            opt.step()

            total_loss += float(loss.item())
            steps += 1
            pbar.set_postfix(loss=total_loss / max(1, steps))

        avg_loss = total_loss / max(1, steps)

        # ---- eval ----
        if ep % eval_every == 0:
            model.eval()
            with torch.no_grad():
                emb_all_eval = model.propagate(A_norm)

            val_metrics = evaluate_loo(emb_all_eval, U, B, offsets, val_gt, train_pos, Ks=(10,20,50))
            score = float(val_metrics["NDCG@10"])

            row = {"run": run_name, "epoch": ep, "loss": avg_loss, **val_metrics}
            history.append(row)

            print(f"[{run_name}] ep={ep:03d} loss={avg_loss:.4f} | "
                  f"Hit@10={val_metrics['Hit@10']:.5f} NDCG@10={val_metrics['NDCG@10']:.5f}")

            if score > best_val + 1e-6:
                best_val = score
                best_state = {"model": model.state_dict(), "epoch": ep, "val_metrics": val_metrics}
                bad = 0
            else:
                bad += 1
                if bad >= patience:
                    print(f"[{run_name}] Early stop at epoch {ep} (best val NDCG@10={best_val:.5f})")
                    break

    return best_state, pd.DataFrame(history)

# ----------------------------
# Run variants
# ----------------------------
all_results = []
all_hist = []

for variant, keep_rel_ids in ablations_big:
    print("\n" + "="*90)
    print("Variant:", variant, "| keep_rel_ids:", len(keep_rel_ids))

    sum_path  = RESULTS_DIR / f"summary_{variant}.json"
    hist_path = RESULTS_DIR / f"hist_{variant}.csv"

    # SKIP if already done
    if sum_path.exists() and hist_path.exists():
        print(f"[SKIP] {variant} already computed")
        with open(sum_path, "r", encoding="utf-8") as f:
            all_results.append(json.load(f))
        continue

    # 1) filter edges
    ei, ew, et = filter_edges_by_rel(edge_index_full, edge_w_full, edge_type_full, keep_rel_ids)
    print("Filtered E:", int(ei.shape[1]))

    # 2) rebuild A_norm
    A_var = build_sparse_norm(ei, ew, num_nodes)
    print("A_var nnz:", int(A_var._nnz()))

    # 3) train
    best_state, hist_df = train_one_run(
        A_norm_cpu=A_var,
        run_name=variant,
        emb_dim=64,
        n_layers=3,
        lr=1e-3,
        epochs=12,
        batch_size=300_000,
        n_neg=1,
        patience=3
    )

    # 4) evaluate TEST
    model = LightGCN(num_nodes=num_nodes, emb_dim=64, n_layers=3).to(DEVICE)
    model.load_state_dict(best_state["model"])
    model.eval()

    with torch.no_grad():
        emb_all = model.propagate(A_var.to(DEVICE))

    test_metrics = evaluate_loo(emb_all, U, B, offsets, test_gt, train_pos, Ks=(10,20,50))

    row = {
        "variant": variant,
        "best_epoch": int(best_state["epoch"]),
        **{f"val_{k}": float(v) for k, v in best_state["val_metrics"].items()},
        **{f"test_{k}": float(v) for k, v in test_metrics.items()},
        "E": int(ei.shape[1]),
        "nnz": int(A_var._nnz()),
        "n_rel_ids": int(len(keep_rel_ids)),
    }

    all_results.append(row)
    all_hist.append(hist_df)

    # save per-variant
    hist_df.to_csv(hist_path, index=False)
    with open(sum_path, "w", encoding="utf-8") as f:
        json.dump(row, f, ensure_ascii=False, indent=2)

    print(f"[DONE] {variant} | TEST Hit@10={test_metrics['Hit@10']:.5f} NDCG@10={test_metrics['NDCG@10']:.5f}")

print("\n[OK] BIG ablation run finished ✅")


Variant: FULL | keep_rel_ids: 11
[SKIP] FULL already computed

Variant: -book_sim | keep_rel_ids: 10
[SKIP] -book_sim already computed

Variant: -book_tag | keep_rel_ids: 9
Filtered E: 10450356
A_var nnz: 10260804


-book_tag | ep 001:   0%|          | 0/17 [00:00<?, ?it/s]

[-book_tag] ep=001 loss=0.6926 | Hit@10=0.01710 NDCG@10=0.00845


-book_tag | ep 002:   0%|          | 0/17 [00:00<?, ?it/s]

[-book_tag] ep=002 loss=0.6863 | Hit@10=0.04738 NDCG@10=0.02541


...

[-book_year] ep=005 loss=0.5712 | Hit@10=0.04657 NDCG@10=0.02623


-book_year | ep 006:   0%|          | 0/17 [00:00<?, ?it/s]

[-book_year] ep=006 loss=0.5156 | Hit@10=0.04656 NDCG@10=0.02615


-book_year | ep 007:   0%|          | 0/17 [00:00<?, ?it/s]

[-book_year] ep=007 loss=0.4739 | Hit@10=0.04676 NDCG@10=0.02619
[-book_year] Early stop at epoch 7 (best val NDCG@10=0.02641)
[DONE] -book_year | TEST Hit@10=0.04742 NDCG@10=0.02630

Variant: ONLY_user_book | keep_rel_ids: 2
[SKIP] ONLY_user_book already computed

Variant: user_book+tag | keep_rel_ids: 4
[SKIP] user_book+tag already computed

[OK] BIG ablation run finished ✅


In [23]:
# ============================
# Summarize ablations (delta vs FULL)
# ============================

res_big = pd.DataFrame(all_results).copy()
if res_big.empty:
    import json
    rows = []
    for p in sorted(RESULTS_DIR.glob("summary_*.json")):
        with open(p, "r", encoding="utf-8") as f:
            rows.append(json.load(f))
    res_big = pd.DataFrame(rows)

# FULL as baseline
full_row = res_big[res_big["variant"] == "FULL"]
if len(full_row) == 0:
    raise ValueError("FULL variant not found in results. Run FULL at least once.")

full = full_row.iloc[0]

for k in ["test_NDCG@10", "test_Hit@10", "test_NDCG@20", "test_Hit@20", "test_NDCG@50", "test_Hit@50"]:
    res_big[f"delta_{k}"] = res_big[k] - float(full[k])

# Sort by main metric
res_big = res_big.sort_values("test_NDCG@10", ascending=False).reset_index(drop=True)

cols = [
    "variant", "n_rel_ids", "best_epoch", "E", "nnz",
    "test_Hit@10", "test_NDCG@10", "delta_test_NDCG@10",
    "test_Hit@20", "test_NDCG@20", "delta_test_NDCG@20",
    "test_Hit@50", "test_NDCG@50", "delta_test_NDCG@50",
]
display(res_big[cols])

print("Baseline FULL:")
print(f"  FULL test_NDCG@10 = {float(full['test_NDCG@10']):.6f}")

,variant,n_rel_ids,best_epoch,E,nnz,test_Hit@10,test_NDCG@10,delta_test_NDCG@10,test_Hit@20,test_NDCG@20,delta_test_NDCG@20,test_Hit@50,test_NDCG@50,delta_test_NDCG@50
0,FULL,NaN,20,11450076,11260518,0.055152,0.029817,0.000000,0.082887,0.036756,0.000000,0.142945,0.048596,0.000000
1,ONLY_user_book,NaN,20,9852768,9852768,0.054983,0.029727,-0.000090,0.082924,0.036715,-0.000041,0.142833,0.048543,-0.000053
2,user_book+tag,NaN,20,10852488,10852482,0.053878,0.029497,-0.000320,0.082194,0.036590,-0.000166,0.141110,0.048221,-0.000375
3,-book_sim,NaN,20,10918914,10918894,0.053279,0.029020,-0.000797,0.080958,0.035954,-0.000802,0.140473,0.047699,-0.000897
4,-book_tag,9.0,12,10450356,10260804,0.049496,0.027478,-0.002339,0.078392,0.034728,-0.002028,0.135361,0.045984,-0.002613
5,-book_lang,9.0,12,11430078,11240520,0.047942,0.026509,-0.003308,0.076145,0.033570,-0.003186,0.132140,0.044649,-0.003948
6,-book_author,9.0,12,11423646,11234102,0.047043,0.026382,-0.003434,0.075883,0.033607,-0.003148,0.131597,0.044615,-0.003981
7,-book_year,9.0,4,11430078,11240520,0.047418,0.026299,-0.003518,0.074778,0.033137,-0.003619,0.129668,0.043999,-0.004598


Baseline FULL:
  FULL test_NDCG@10 = 0.029817


In [24]:
# ============================
# Save final results table
# ============================
out_csv = RESULTS_DIR / "ablation_summary_BIG.csv"
res_big.to_csv(out_csv, index=False)
print("[OK] saved:", out_csv)

# Дополнительно — компактная "витринная" таблица
out_csv_small = RESULTS_DIR / "ablation_summary_BIG_compact.csv"
compact_cols = [
    "variant",
    "test_Hit@10", "test_NDCG@10",
    "test_Hit@20", "test_NDCG@20",
    "test_Hit@50", "test_NDCG@50",
    "delta_test_NDCG@10",
    "E", "nnz", "best_epoch"
]
res_big[compact_cols].to_csv(out_csv_small, index=False)
print("[OK] saved:", out_csv_small)

[OK] saved: D:\ML\GNN\graph_recsys\artifacts\v2_proper\ablation_runs\ablation_summary_BIG.csv
[OK] saved: D:\ML\GNN\graph_recsys\artifacts\v2_proper\ablation_runs\ablation_summary_BIG_compact.csv


## Ablation Study Summary (Graph3 + LightGCN)

In this notebook, we conducted a systematic **ablation study** on the enriched **Graph3** constructed from the **Goodbooks-10k** dataset in order to quantify the contribution of different relation types to recommendation quality.

### Experimental Setup
- **Model:** LightGCN  
- **Graph:** Unified node space (users, books, tags, authors, language, year_bin)  
- **Task:** Link prediction / recommendation  
- **Evaluation:** Leave-One-Out (LOO)  
- **Metrics:** Hit@K, NDCG@K  
- **Baseline:** FULL graph with all relation types enabled  

### Ablation Variants
We evaluated the impact of removing or isolating specific relation groups:
- Removal of individual relations (`-book_tag`, `-book_author`, `-book_lang`, `-book_year`, `-book_sim`);
- Simplified graph structures (`ONLY_user_book`, `user_book+tag`);
- All variants were compared against the FULL graph baseline.

### Key Results (TEST NDCG@10)

| Variant | TEST NDCG@10 | Δ vs FULL |
|------|-------------|-----------|
| **FULL** | **0.0298** | baseline |
| ONLY_user_book | 0.0297 | −0.0001 |
| user_book+tag | 0.0295 | −0.0003 |
| −book_sim | 0.0290 | −0.0008 |
| **−book_tag** | 0.0275 | −0.0023 |
| **−book_lang** | 0.0265 | −0.0033 |
| **−book_author** | 0.0264 | −0.0034 |
| **−book_year** | 0.0263 | −0.0035 |

### Main Observations

1. **User–Book interactions carry the dominant signal**  
   A graph containing only user–book edges achieves almost the same performance as the full graph.

2. **Book–Book similarity provides a small but consistent benefit**  
   Removing TF-IDF–based similarity edges leads to a measurable degradation in ranking quality.

3. **Content-based meta-relations are critical when used together**  
   Removing relations related to:
   - publication year,
   - authors,
   - language,
   - tags  
   results in a substantial performance drop (up to −0.0035 NDCG@10).

4. **Individual content signals do not work well in isolation**  
   The `user_book+tag` variant underperforms the full graph, indicating that tags alone introduce noise unless supported by additional contextual relations.

5. **LightGCN does not explicitly model heterogeneity**  
   All neighbors are aggregated uniformly, without:
   - relation-specific parameters,
   - attention mechanisms,
   - explicit node or edge typing.  

   As a result, the model fails to fully exploit the rich heterogeneous structure of Graph3.

### Final Conclusion

This ablation study demonstrates that:
- the enriched Graph3 contains meaningful and complementary signals;
- **LightGCN acts as the main bottleneck** when applied to heterogeneous graphs;
- further improvements are unlikely to come from adding more edges or features, but rather from **more expressive, relation-aware architectures**.

### Next Steps

The natural continuation of this work is to transition to models that explicitly account for relation types, such as:
- **Relational GCN (R-GCN) / HeteroConv**
- **Heterogeneous Graph Transformer (HGT)**

These models are better suited to leverage the full potential of Graph3 and to assess whether its rich structure can translate into meaningful gains in recommendation quality.